# Trend Screener

In [47]:
pip install arch scipy yfinance pandas numpy QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 2.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
import datetime as dt
import pandas as pd
import yfinance as yf
import numpy as np
from arch import arch_model
from scipy.stats import norm
import QuantLib as ql

In [78]:
symbol = "AAPL"

def process_expiration(exp_td_str):
    options = tk.option_chain(exp_td_str)
    
    calls = options.calls
    puts = options.puts
    
    # Add optionType column
    calls['optionType'] = 'C'
    puts['optionType'] = 'P'
    
    # Merge calls and puts into a single dataframe
    exp_data = pd.concat(objs=[calls, puts], ignore_index=True)
    
    return exp_data


tk = yf.Ticker(symbol)
expirations = tk.options

# Create empty DataFrame, then add individual expiration data to it
data = pd.DataFrame()

for exp_td_str in expirations[:1]:
    exp_data = process_expiration(exp_td_str)
    data = pd.concat(objs=[data, exp_data], ignore_index=True)
    
# Add underlyingSymbol column
data['underlyingSymbol'] = symbol

print(data)

         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   AAPL241004C00100000 2024-09-16 18:59:10+00:00   100.0     116.70  127.70   
1   AAPL241004C00105000 2024-09-16 18:59:10+00:00   105.0     111.70  122.45   
2   AAPL241004C00125000 2024-09-27 17:21:27+00:00   125.0     103.05  102.70   
3   AAPL241004C00135000 2024-09-26 19:31:38+00:00   135.0      92.86   91.80   
4   AAPL241004C00140000 2024-09-26 15:15:04+00:00   140.0      87.39   87.75   
..                  ...                       ...     ...        ...     ...   
86  AAPL241004P00250000 2024-09-27 19:37:30+00:00   250.0      22.30   21.60   
87  AAPL241004P00255000 2024-09-27 18:56:34+00:00   255.0      26.92   27.00   
88  AAPL241004P00265000 2024-09-27 19:45:08+00:00   265.0      37.35   36.90   
89  AAPL241004P00275000 2024-09-19 19:50:00+00:00   275.0      45.95   46.85   
90  AAPL241004P00300000 2024-09-24 15:12:51+00:00   300.0      72.55   71.85   

       ask    change  percentChange  vo

In [91]:

def price_heston_option(
    spot_price, 
    strike_price, 
    risk_free_rate, 
    dividend_yield, 
    initial_volatility, 
    expiry, 
    kappa, 
    theta, 
    sigma, 
    rho, 
    option_type='call'
):
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    rate_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual360())
    )
    dividend_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), ql.Actual360())
    )

    heston_process = ql.HestonProcess(
        rate_handle,  # Risk-free rate
        dividend_handle,  # Dividend yield
        spot_handle,  # Initial stock price
        initial_volatility ** 2,  # Initial variance (square of initial volatility)
        kappa,  # Mean reversion speed
        theta,  # Long-term variance
        sigma,  # Volatility of volatility
        rho  # Correlation
    )
    heston_model = ql.HestonModel(heston_process)

    expiry_date = ql.Date().todaysDate() + ql.Period(int(expiry), ql.Days)
    exercise = ql.EuropeanExercise(expiry_date)
    payoff_type = ql.Option.Call if option_type.lower() == 'call' else ql.Option.Put
    payoff = ql.PlainVanillaPayoff(payoff_type, strike_price)
    european_option = ql.VanillaOption(payoff, exercise)

    engine = ql.AnalyticHestonEngine(heston_model)
    european_option.setPricingEngine(engine)

    option_price = european_option.NPV()
    return option_price

option_price = price_heston_option(
    spot_price=2.82,
    strike_price=3,
    risk_free_rate=0.05,
    dividend_yield=0.0,
    initial_volatility=0.72,
    expiry=20,
    kappa=1.0,
    theta=1.0,
    sigma=0.1,
    rho=-0.5,
    option_type='put'
)

print(f"The European option price using Heston model: {option_price:.4f}")


The European option price using Heston model: 0.2961


In [105]:
def fetch_and_calculate_garch_volatility(ticker, start_date, end_date, p=1, q=1):
    """
    Fetch historical stock price data, calculate daily returns, and compute GARCH volatility.
    
    Parameters:
    - ticker: The stock ticker symbol (e.g., 'AAPL').
    - start_date: The start date for fetching data (format 'YYYY-MM-DD').
    - end_date: The end date for fetching data (format 'YYYY-MM-DD').
    - p: The order of the GARCH term.
    - q: The order of the ARCH term.
    
    Returns:
    - A pandas Series of estimated conditional volatilities.
    """
    
    # Fetch historical stock price data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Calculate daily returns
    returns = stock_data['Adj Close'].pct_change().dropna()

    # Fit GARCH model and calculate volatility
    model = arch_model(returns, vol='Garch', p=p, q=q, rescale=True)
    model_fit = model.fit(disp="off")
    daily_volatility = model_fit.conditional_volatility

    # Convert to annual volatility (assuming approximately 252 trading days)
    annual_volatility = daily_volatility * np.sqrt(252) / 10

    annual_volatility.std()
    
    return annual_volatility

ticker = 'CLOV'
start_date = '2024-06-27'
end_date = '2024-09-27'
    
volatilities = fetch_and_calculate_garch_volatility(ticker, start_date, end_date, p=1, q=1)
print(volatilities)

[*********************100%%**********************]  1 of 1 completed

Date
2023-09-28    0.737346
2023-09-29    0.678313
2023-10-02    0.630495
2023-10-03    0.588064
2023-10-04    0.589789
                ...   
2024-09-20    0.705578
2024-09-23    0.710067
2024-09-24    0.653299
2024-09-25    0.624334
2024-09-26    0.608075
Name: cond_vol, Length: 251, dtype: float64


In [73]:
import QuantLib as ql

# Option parameters
spot_price = 2.82  # Current stock price
strike_price = 3  # Option strike price
risk_free_rate = 0.05  # Risk-free interest rate
dividend_yield = 0.0  # Dividend yield
initial_volatility = 0.61  # Volatility of the underlying asset
expiry = 20  # Time to expiry in years (1 year)

# Set the evaluation date
today = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = today

# Create the underlying asset and its parameters
initial_value = ql.QuoteHandle(ql.SimpleQuote(spot_price))
risk_free_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual365Fixed()))
dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), ql.Actual365Fixed()))
volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(initial_volatility)), ql.Actual365Fixed()))

# Create the Black-Scholes Merton process
process = ql.BlackScholesMertonProcess(initial_value, dividend_ts, risk_free_ts, volatility)

# Define the number of time steps and paths for the Monte Carlo simulation
steps = 100
num_paths = 100000
rng = "pseudorandom"  # could use "lowdiscrepancy"

# Set up the Monte Carlo engine
engine = ql.MCEuropeanEngine(process, rng, steps, requiredSamples=num_paths)

# Create the European option
expiry_date = today + ql.Period(int(expiry), ql.Days)  # Expiry date
exercise = ql.EuropeanExercise(expiry_date)
payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
european_option = ql.VanillaOption(payoff, exercise)

# Set the pricing engine
european_option.setPricingEngine(engine)

# Calculate the option price
option_price = european_option.NPV()
print(f"The European option price using Monte Carlo simulation: {option_price:.4f}")


The European option price using Monte Carlo simulation: 0.1521
